In [ ]:
%pip install sklearn-hierarchical-classification
!pip install scikit-multilearn
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.datasets import make_multilabel_classification
from sklearn_hierarchical_classification.classifier import HierarchicalClassifier
from sklearn_hierarchical_classification.constants import ROOT
import json
import os
import numpy as np
import networkx as nx
from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.metrics import accuracy_score
from skmultilearn.adapt import MLkNN
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn_hierarchical_classification.classifier import HierarchicalClassifier
from sklearn_hierarchical_classification.constants import ROOT
from networkx import DiGraph
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix
import re

  Preparing metadata (setup.py) ... done
  Created wheel for sklearn-hierarchical-classification: filename=sklearn_hierarchical_classification-1.3.2-py3-none-any.whl size=16916 sha256=0946247f04eac048a7626b0f45a4b23332772680c4d21a2b2e4017b9140b2802
  Stored in directory: /root/.cache/pip/wheels/87/09/e3/1619c1a173b533be37bda720b5f5aaa719e116d66ff7c01a12
Successfully built sklearn-hierarchical-classification
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 2.8 MB/s eta 0:00:00


In [ ]:
"""
!pip install auto-sklearn
from pprint import pprint

import pandas as pd
import sklearn.metrics
from sklearn.datasets import fetch_20newsgroups

import autosklearn.classification
"""

# serie di 4 funzioni per la suddivisione del dataset secondo etichette
def find_descendants(graph, labels):
    descendants = set()

    def dfs(node):
        descendants.add(node)
        [dfs(child) for child in graph.successors(node)]

    [dfs(label) for label in labels]
    return descendants

def filter_matrix_by_labels(X, y, labels, graph):
    descendants_set = find_descendants(graph, labels)

    filtered_X, filtered_labels = zip(*[(X[i], list(set(labels_list) & descendants_set)) for i, labels_list in enumerate(y) if set(labels_list) & descendants_set])

    return np.vstack(filtered_X), list(filtered_labels)

def find_ancestor_in_labels(graph, node, target_labels):
    ancestors = set()

    def dfs(current):
        ancestors.add(current)
        [dfs(pred) for pred in graph.predecessors(current)]

    [dfs(predecessor) for predecessor in graph.predecessors(node)]

    common_ancestors = ancestors & set(target_labels)
    return list(common_ancestors)[0] if common_ancestors else None

def transform_Y_with_ancestors(graph, Y, target_labels):
    return [list(set([find_ancestor_in_labels(graph, label, target_labels) or label for label in labels_list])) for labels_list in Y]
###############################


def fit_classifier(X_Train, y_Train, G, labels_to_check, classifier, vectorizer):
    X, Y = filter_matrix_by_labels(X_train, y_train, labels_to_check, G)
    new_Y = transform_Y_with_ancestors(G, Y, labels_to_check)
    Y = Binarizzatore_etichette(labels_to_check, new_Y)
    X_tfidf = vectorizer.transform(X.flatten().tolist())

    return classifier.fit(X_tfidf, Y)


def Binarizzatore_etichette(lista_stringhe, lista_liste):
    # Inizializza la matrice con zeri
    matrice = [[0] * len(lista_stringhe) for _ in range(len(lista_liste))]

    # Riempie la matrice con 1 dove una stringa è presente nella lista di liste
    for i, lista in enumerate(lista_liste):
        for j, stringa in enumerate(lista_stringhe):
            if stringa in lista:
                matrice[i][j] = 1

    return matrice


def read_json_file(file_path):
    if os.path.exists(file_path):
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
        return data
    else:
        return None


def check_errors(graph, binary_vector, predicted_labels, true_labels):
    for pred_label, pred_value in zip(predicted_labels, binary_vector):
        if pred_value == 1 and not any(nx.has_path(graph.reverse(), label, pred_label) for label in true_labels):
            return False
        elif pred_value == 0 and pred_label in true_labels:
            return False

    return True  # Se non ci sono errori, restituisce True



In [ ]:

# Construct the graph
G = DiGraph()
G.add_edge(ROOT, "Free")
G.add_edge(ROOT, "Persuasion")
G.add_edge("Persuasion", "Logos")
G.add_edge("Logos", "Repetition")
G.add_edge("Logos", "Obfuscation, Intentional vagueness, Confusion")
G.add_edge("Logos", "Reasoning")
G.add_edge("Logos", "Justification")
G.add_edge('Justification', "Slogans")
G.add_edge('Justification', "Bandwagon")
G.add_edge('Justification', "Appeal to authority")
G.add_edge('Justification', "Flag-waving")
G.add_edge('Justification', "Appeal to fear/prejudice")
G.add_edge('Reasoning', "Simplification")
G.add_edge('Simplification', "Causal Oversimplification")
G.add_edge('Simplification', "Black-and-white Fallacy/Dictatorship")
G.add_edge('Simplification', "Thought-terminating cliché")
G.add_edge('Reasoning', "Distraction")
G.add_edge('Distraction', "Misrepresentation of Someone's Position (Straw Man)")
G.add_edge('Distraction', "Presenting Irrelevant Data (Red Herring)")
G.add_edge('Distraction', "Whataboutism")
G.add_edge("Persuasion", "Ethos")
G.add_edge('Ethos', "Appeal to authority")
G.add_edge('Ethos', "Glittering generalities (Virtue)")
G.add_edge('Ethos', "Bandwagon")
G.add_edge('Ethos', "Ad Hominem")
G.add_edge('Ad Hominem', "Doubt")
G.add_edge('Ad Hominem', "Name calling/Labeling")
G.add_edge('Ad Hominem', "Smears")
G.add_edge('Ad Hominem', "Reductio ad hitlerum")
G.add_edge('Ad Hominem', "Whataboutism")
G.add_edge("Persuasion", "Pathos")
G.add_edge('Pathos', "Exaggeration/Minimisation")
G.add_edge('Pathos', "Loaded Language")
G.add_edge('Pathos', "Appeal to fear/prejudice")
G.add_edge('Pathos', "Flag-waving")


np.set_printoptions(threshold=np.inf)
# Leggi train.json
train_data = read_json_file("train.json")

# Leggi validation.json
validation_data = read_json_file("validation.json")

test_data = read_json_file("dev_subtask1_en.json")


# Inizializza le liste per i dati di addestramento
texts_train, labels_train = [], []
texts_validation, labels_validation = [], []
texts_test, labels_test = [], []
texts_test_vero, labels_test_vero = [], []
if train_data:
    texts_train = [element['text'] for element in train_data]
    labels_train = [element['labels'] if element['labels'] else ['Free'] for element in train_data]

# Estre x e y dai dataset se esistono
if validation_data:
    texts_validation = [element['text'] for element in validation_data]
    labels_validation = [element['labels'] if element['labels'] else ['Free'] for element in validation_data]

if test_data:
    texts_test = [element['text'] for element in test_data]
    labels_test = [element['labels'] if element['labels'] else ['Free'] for element in test_data]

# Unisce i dati di addestramento e validation
texts = texts_train + texts_validation
labels = labels_train + labels_validation


"""
# Divide dataset in set di addestramento e test
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)  # , stratify=results
"""
####### nuova sezione per avere validation.json come gold
X_test = texts_test
y_test = labels_test
X_train = texts
y_train = labels


from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(X_train)
X_test = vectorizer.transform(X_test)

# Definizione dei classificatori
Free_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
Persuasion_classifier = MultiOutputClassifier(RandomForestClassifier(n_estimators=100, random_state=42))
Ethos_classifier = MultiOutputClassifier(RandomForestClassifier(n_estimators=100, random_state=42))
Ad_hominem_classifier = MultiOutputClassifier(RandomForestClassifier(n_estimators=100, random_state=42))
Pathos_classifier = MultiOutputClassifier(RandomForestClassifier(n_estimators=100, random_state=42))
Logos_classifier = MultiOutputClassifier(RandomForestClassifier(n_estimators=100, random_state=42))
Justification_classifier = MultiOutputClassifier(RandomForestClassifier(n_estimators=100, random_state=42))
Reasoning_classifier = MultiOutputClassifier(RandomForestClassifier(n_estimators=100, random_state=42))
Distraction_classifier = MultiOutputClassifier(RandomForestClassifier(n_estimators=100, random_state=42))
Simplification_classifier = MultiOutputClassifier(RandomForestClassifier(n_estimators=100, random_state=42))

#addestramento dei classificatori tramite apposita funzione che gli permette di usare solo il sottoinsieme del dataset che gli serve
Free_classifier = fit_classifier(X_train, y_train, G, ["Free", "Persuasion"], Free_classifier, vectorizer)
Persuasion_classifier = fit_classifier(X_train, y_train, G, ["Ethos", "Pathos", "Logos"], Persuasion_classifier, vectorizer)
Ethos_classifier = fit_classifier(X_train, y_train, G, ["Ad Hominem", "Bandwagon", "Glittering generalities (Virtue)" , "Appeal to authority"], Ethos_classifier, vectorizer)
Ad_hominem_classifier = fit_classifier(X_train, y_train, G, ["Doubt", "Name calling/Labeling", "Smears", "Reductio ad hitlerum","Whataboutism"], Ad_hominem_classifier, vectorizer)
Pathos_classifier = fit_classifier(X_train, y_train, G, ["Exaggeration/Minimisation", "Loaded Language", "Appeal to fear/prejudice", "Flag-waving"], Pathos_classifier, vectorizer)
Logos_classifier = fit_classifier(X_train, y_train, G, ["Repetition", "Obfuscation, Intentional vagueness, Confusion", "Reasoning", "Justification"], Logos_classifier, vectorizer)
Justification_classifier = fit_classifier(X_train, y_train, G, ["Slogans", "Bandwagon", "Appeal to authority", "Flag-waving", "Appeal to fear/prejudice"], Justification_classifier, vectorizer)
Reasoning_classifier = fit_classifier(X_train, y_train, G, ["Simplification", "Distraction"], Reasoning_classifier, vectorizer)
Distraction_classifier = fit_classifier(X_train, y_train, G, ["Whataboutism","Misrepresentation of Someone's Position (Straw Man)","Presenting Irrelevant Data (Red Herring)"], Distraction_classifier, vectorizer)
Simplification_classifier = fit_classifier(X_train, y_train, G, ["Causal Oversimplification", "Black-and-white Fallacy/Dictatorship", "Thought-terminating cliché"], Simplification_classifier, vectorizer)



In [ ]:
y_pred = []

# primo carattere conta il numero di errori, il seocndo il numero di volte che viene usato il classificatore
counter_Free = [0, X_test.shape[0]]
counter_Persuasion = [0, 0]
counter_Ethos = [0, 0]
counter_Ad_hominem = [0, 0]
counter_Pathos = [0, 0]
counter_Logos = [0, 0]
counter_Justification = [0, 0]
counter_Reasoning = [0, 0]
counter_Distraction = [0, 0]
counter_Simplification = [0, 0]

for i in range(X_test.shape[0]):
    y_pred.append([])

    #probabilities = Free_classifier.predict_proba(X_test[i])
    prediction = Free_classifier.predict(X_test[i])
    counter_Free[0] += not check_errors(G, prediction[0], ["Free", "Persuasion"], y_test[i])

    # Free
    if prediction[0, 0] == 1:
        #y_pred[i].append("Free")
        continue

    # Persuasion
    if prediction[0, 1] == 1:
        prediction = Persuasion_classifier.predict(X_test[i])
        counter_Persuasion[1] += 1
        counter_Persuasion[0] += not check_errors(G, prediction[0], ["Ethos", "Pathos", "Logos"], y_test[i])

        #Ethos
        if prediction[0, 0] == 1:
            prediction1 = Ethos_classifier.predict(X_test[i])
            counter_Ethos[1] += 1
            counter_Ethos[0] += not check_errors(G, prediction1[0], ["Ad Hominem", "Bandwagon", "Glittering generalities (Virtue)" , "Appeal to authority"], y_test[i])

            # Ad Hominem
            if prediction1[0, 0] == 1:
                prediction8 = Ad_hominem_classifier.predict(X_test[i])
                counter_Ad_hominem[1] += 1
                counter_Ad_hominem[0] += not check_errors(G, prediction8[0], ["Doubt", "Name calling/Labeling", "Smears", "Reductio ad hitlerum","Whataboutism"], y_test[i])
                if prediction8[0, 0] == 1:
                    y_pred[i].append("Doubt")
                if prediction8[0, 1] == 1:
                    y_pred[i].append("Name calling/Labeling")
                if prediction8[0, 2] == 1:
                    y_pred[i].append("Smears")
                if prediction8[0, 3] == 1:
                    y_pred[i].append("Reductio ad hitlerum")
                if prediction8[0, 4] == 1:
                    y_pred[i].append("Whataboutism")
                if all(prediction8[0, i] == 0 for i in range(5)):
                    y_pred[i].append("Ad Hominem")

            if prediction1[0, 1] == 1:
                y_pred[i].append("Bandwagon")
            if prediction1[0, 2] == 1:
                y_pred[i].append("Glittering generalities (Virtue)")
            if prediction1[0, 3] == 1:
                y_pred[i].append("Appeal to authority")
            if all(prediction1[0, i] == 0 for i in range(4)):
                y_pred[i].append("Ethos")
        #Pathos
        if prediction[0, 1] == 1:
            prediction2 = Pathos_classifier.predict(X_test[i])
            counter_Pathos[1] += 1
            counter_Pathos[0] += not check_errors(G, prediction2[0], ["Exaggeration/Minimisation", "Loaded Language", "Appeal to fear/prejudice", "Flag-waving"], y_test[i])
            if prediction2[0, 0] == 1:
                y_pred[i].append("Exaggeration/Minimisation")
            if prediction2[0, 1] == 1:
                y_pred[i].append("Loaded Language")
            if prediction2[0, 2] == 1:
                y_pred[i].append("Appeal to fear/prejudice")
            if prediction2[0, 3] == 1:
                y_pred[i].append("Flag-waving")
            if all(prediction2[0, i] == 0 for i in range(4)):
                y_pred[i].append("Pathos")
        #Logos
        if prediction[0, 2] == 1:
            prediction3 = Logos_classifier.predict(X_test[i])
            counter_Logos[1] += 1
            counter_Logos[0] += not check_errors(G, prediction3[0], ["Repetition", "Obfuscation, Intentional vagueness, Confusion", "Reasoning", "Justification"], y_test[i])
            if prediction3[0, 0] == 1:
                y_pred[i].append("Repetition")
            if prediction3[0, 1] == 1:
                y_pred[i].append("Obfuscation, Intentional vagueness, Confusion")

            # Reasoning
            if prediction3[0, 2] == 1:
                prediction4 = Reasoning_classifier.predict(X_test[i])
                counter_Reasoning[1] += 1
                counter_Reasoning[0] += not check_errors(G, prediction4[0], ["Simplification", "Distraction"], y_test[i])

                #Simplification
                if prediction4[0, 0] == 1:
                    prediction6 = Simplification_classifier.predict(X_test[i])
                    counter_Simplification[1] += 1
                    counter_Simplification[0] += not check_errors(G, prediction6[0], ["Causal Oversimplification", "Black-and-white Fallacy/Dictatorship", "Thought-terminating cliché"], y_test[i])
                    if prediction6[0, 0] == 1:
                        y_pred[i].append("Causal Oversimplification")
                    if prediction6[0, 1] == 1:
                        y_pred[i].append("Black-and-white Fallacy/Dictatorship")
                    if prediction6[0, 2] == 1:
                        y_pred[i].append("Thought-terminating cliché")
                    if all(prediction6[0, i] == 0 for i in range(3)):
                        y_pred[i].append("Simplification")

                #Distraction
                if prediction4[0, 1] == 1:
                    prediction7 = Distraction_classifier.predict(X_test[i])
                    counter_Distraction[1] += 1
                    counter_Distraction[0] += not check_errors(G, prediction7[0], ["Whataboutism","Misrepresentation of Someone's Position (Straw Man)","Presenting Irrelevant Data (Red Herring)"], y_test[i])
                    if prediction7[0, 0] == 1:
                        y_pred[i].append("Whataboutism")
                    if prediction7[0, 1] == 1:
                        y_pred[i].append("Misrepresentation of Someone's Position (Straw Man)")
                    if prediction7[0, 2] == 1:
                        y_pred[i].append("Presenting Irrelevant Data (Red Herring)")
                    if all(prediction7[0, i] == 0 for i in range(3)):
                        y_pred[i].append("Distraction")

            #Justification
            if prediction3[0, 3] == 1:
                prediction5 = Justification_classifier.predict(X_test[i])
                counter_Justification[1] += 1
                counter_Justification[0] += not check_errors(G, prediction5[0], ["Slogans", "Bandwagon", "Appeal to authority", "Flag-waving", "Appeal to fear/prejudice"], y_test[i])
                if prediction5[0, 0] == 1:
                    y_pred[i].append("Slogans")
                if prediction5[0, 1] == 1:
                    y_pred[i].append("Bandwagon")
                if prediction5[0, 2] == 1:
                    y_pred[i].append("Appeal to authority")
                if prediction5[0, 3] == 1:
                    y_pred[i].append("Flag-waving")
                if prediction5[0, 4] == 1:
                    y_pred[i].append("Appeal to fear/prejudice")
                if all(prediction5[0, i] == 0 for i in range(5)):
                    y_pred[i].append("Justification")

            if all(prediction3[0, i] == 0 for i in range(4)):
                y_pred[i].append("Logos")

        #if all(prediction[0, i] == 0 for i in range(3)):
        #    y_pred[i].append("Persuasion")


# Per evitare ripetizioni delle foglie raggiunguibili da più percorsi
for i in range(len(y_pred)):
    y_pred[i] = list(set(y_pred[i]))


In [ ]:
# copia fatta nel main
def check_errors(graph, binary_vector, predicted_labels, true_labels):
    for pred_label, pred_value in zip(predicted_labels, binary_vector):
        if pred_value == 1 and not any(nx.has_path(graph.reverse(), label, pred_label) for label in true_labels):
            print("Errore nelle etichette predette:", pred_label)
            return False
        elif pred_value == 0 and pred_label in true_labels:
            print("Errore nelle etichette predette:", pred_label)
            return False

    return True  # Se non ci sono errori, restituisce True


a = check_errors(G, [0,1], ["Free", "Persuasion"], ["Free"])
print(a)



Errore nelle etichette predette: Free
False


In [ ]:
def print_results(counters):
    categories = ["Free", "Persuasion", "Ethos", "Ad Hominem", "Pathos", "Logos",
                  "Justification", "Reasoning", "Distraction", "Simplification"]

    print("{:<20} {:<10} {:<10}".format("Category", "Errors", "Usage"))
    print("-" * 40)

    for category, counter in zip(categories, counters):
        print("{:<20} {:<10} {:<10}".format(category, counter[0], counter[1]))

# Esempio di utilizzo
counters = [counter_Free, counter_Persuasion, counter_Ethos, counter_Ad_hominem,
            counter_Pathos, counter_Logos, counter_Justification, counter_Reasoning,
            counter_Distraction, counter_Simplification]

print_results(counters)

In [ ]:
import pdb
import json
import logging.handlers
import argparse
import os
import numpy as np
from sklearn.metrics import f1_score
from sklearn.preprocessing import MultiLabelBinarizer
from networkx import DiGraph, relabel_nodes, all_pairs_shortest_path_length
from sklearn_hierarchical_classification.constants import ROOT
from sklearn_hierarchical_classification.metrics import h_fbeta_score, h_recall_score, h_precision_score, \
    fill_ancestors, multi_labeled
import sys
sys.path.append('.')


G = DiGraph()
G.add_edge(ROOT, "Logos")
G.add_edge("Logos", "Repetition")
G.add_edge("Logos", "Obfuscation, Intentional vagueness, Confusion")
G.add_edge("Logos", "Reasoning")
G.add_edge("Logos", "Justification")
G.add_edge('Justification', "Slogans")
G.add_edge('Justification', "Bandwagon")
G.add_edge('Justification', "Appeal to authority")
G.add_edge('Justification', "Flag-waving")
G.add_edge('Justification', "Appeal to fear/prejudice")
G.add_edge('Reasoning', "Simplification")
G.add_edge('Simplification', "Causal Oversimplification")
G.add_edge('Simplification', "Black-and-white Fallacy/Dictatorship")
G.add_edge('Simplification', "Thought-terminating cliché")
G.add_edge('Reasoning', "Distraction")
G.add_edge('Distraction', "Misrepresentation of Someone's Position (Straw Man)")
G.add_edge('Distraction', "Presenting Irrelevant Data (Red Herring)")
G.add_edge('Distraction', "Whataboutism")
G.add_edge(ROOT, "Ethos")
G.add_edge('Ethos', "Appeal to authority")
G.add_edge('Ethos', "Glittering generalities (Virtue)")
G.add_edge('Ethos', "Bandwagon")
G.add_edge('Ethos', "Ad Hominem")
G.add_edge('Ethos', "Transfer")
G.add_edge('Ad Hominem', "Doubt")
G.add_edge('Ad Hominem', "Name calling/Labeling")
G.add_edge('Ad Hominem', "Smears")
G.add_edge('Ad Hominem', "Reductio ad hitlerum")
G.add_edge('Ad Hominem', "Whataboutism")
G.add_edge(ROOT, "Pathos")
G.add_edge('Pathos', "Exaggeration/Minimisation")
G.add_edge('Pathos', "Loaded Language")
G.add_edge('Pathos', "Appeal to (Strong) Emotions")
G.add_edge('Pathos', "Appeal to fear/prejudice")
G.add_edge('Pathos', "Flag-waving")
G.add_edge('Pathos', "Transfer")

KEYS = ['id','labels']
logger = logging.getLogger("subtask_1_2a_scorer")
ch = logging.StreamHandler(sys.stdout)
ch.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)
logger.setLevel(logging.INFO)
#logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.INFO)

def _read_gold_and_pred(pred_fpath, gold_fpath):
  """
  Read gold and predicted data.
  :param pred_fpath: a json file with predictions,
  :param gold_fpath: the original annotated gold file.
  :return: {id:pred_labels} dict; {id:gold_labels} dict
  """

  gold_labels = {}
  with open(gold_fpath, encoding='utf-8') as gold_f:
    gold = json.load(gold_f)
    for obj in gold:
      gold_labels[obj['id']] = obj['labels']

  pred_labels = {}
  with open(pred_fpath, encoding='utf-8') as pred_f:
    pred = json.load(pred_f)
    for obj in pred:
      pred_labels[obj['id']] = obj['labels']

  if set(gold_labels.keys()) != set(pred_labels.keys()):
      logger.error('There are either missing or added examples to the prediction file. Make sure you only have the gold examples in the prediction file.')
      raise ValueError('There are either missing or added examples to the prediction file. Make sure you only have the gold examples in the prediction file.')

  return pred_labels, gold_labels

#F1
def _h_fbeta_score(y_true, y_pred, class_hierarchy, beta=1., root=ROOT):
    hP = _h_precision_score(y_true, y_pred, class_hierarchy, root=root)
    hR = _h_recall_score(y_true, y_pred, class_hierarchy, root=root)
    return (1. + beta ** 2.) * hP * hR / (beta ** 2. * hP + hR)

def _fill_ancestors(y, graph, root, copy=True):
    y_ = y.copy() if copy else y
    paths = all_pairs_shortest_path_length(graph.reverse(copy=False))
    for target, distances in paths:
        if target == root:
            continue
        ix_rows = np.where(y[:, target] > 0)[0]
        ancestors = list(filter(lambda x: x != ROOT,distances.keys()))
        y_[tuple(np.meshgrid(ix_rows, ancestors))] = 1
    graph.reverse(copy=False)
    return y_
def _h_recall_score(y_true, y_pred, class_hierarchy, root=ROOT):
    y_true_ = _fill_ancestors(y_true, graph=class_hierarchy, root=root)
    y_pred_ = _fill_ancestors(y_pred, graph=class_hierarchy, root=root)

    ix = np.where((y_true_ != 0) & (y_pred_ != 0))

    true_positives = len(ix[0])
    all_positives = np.count_nonzero(y_true_)

    return true_positives / all_positives

def _h_precision_score(y_true, y_pred, class_hierarchy, root=ROOT):
    y_true_ = _fill_ancestors(y_true, graph=class_hierarchy, root=root)
    y_pred_ = _fill_ancestors(y_pred, graph=class_hierarchy, root=root)

    ix = np.where((y_true_ != 0) & (y_pred_ != 0))

    true_positives = len(ix[0])
    all_results = np.count_nonzero(y_pred_)

    return true_positives / all_results


def evaluate_h(pred_fpath, gold_fpath):
    pred_labels, gold_labels = _read_gold_and_pred(pred_fpath, gold_fpath)

    gold = []
    pred = []
    for id in gold_labels:
        gold.append(gold_labels[id])
        pred.append(pred_labels[id])
    with multi_labeled(gold, pred, G) as (gold_, pred_, graph_):
        return  _h_precision_score(gold_, pred_,graph_), _h_recall_score(gold_, pred_,graph_), _h_fbeta_score(gold_, pred_,graph_)

prec_h, rec_h, f1_h = evaluate_h("pred4.json", "dev_subtask1_en.json")
print("f1_h={:.5f}\tprec_h={:.5f}\trec_h={:.5f}".format(f1_h, prec_h, rec_h))

f1_h=0.51483	prec_h=0.52837	rec_h=0.50198


In [ ]:


#aaaa = _read_gold_and_pred("dev_subtask1_en.json")

In [ ]:
from collections import Counter
# Dopo aver creato e popolato tutte le liste y_pred
element_count = Counter(len(lst) for lst in y_pred)

for num_elements in range(11):
    count = element_count.get(num_elements, 0)
    print(f"Numero di liste con {num_elements} elemento/i: {count}")
"""
    if num_elements == 0:
        print("Liste con zero elementi:")
        for idx, lst in enumerate(y_pred):
            if len(lst) == 0:
                print(f"Lista {idx + 1}: {lst}")
"""

Numero di liste con 0 elemento/i: 151
Numero di liste con 1 elemento/i: 315
Numero di liste con 2 elemento/i: 254
Numero di liste con 3 elemento/i: 168
Numero di liste con 4 elemento/i: 81
Numero di liste con 5 elemento/i: 23
Numero di liste con 6 elemento/i: 5
Numero di liste con 7 elemento/i: 2
Numero di liste con 8 elemento/i: 1
Numero di liste con 9 elemento/i: 0
Numero di liste con 10 elemento/i: 0


'\n    if num_elements == 0:\n        print("Liste con zero elementi:")\n        for idx, lst in enumerate(y_pred):\n            if len(lst) == 0:\n                print(f"Lista {idx + 1}: {lst}")\n'

In [ ]:
import json
from collections import Counter

# Carica le previsioni dal file JSON (assumendo che sia una lista di dizionari con la chiave "label")
with open('pred.json', 'r') as file:
    data = json.load(file)

# Conta il numero di elementi in ciascuna lista "label"
element_count = Counter(len(item["label"]) for item in data)

# Stampa il conteggio per ciascun numero di elementi
for num_elements in range(11):
    count = element_count.get(num_elements, 0)
    print(f"Numero di liste 'label' con {num_elements} elemento/i: {count}")

KeyError: 'label'

In [ ]:
import copy

# Creazione di copie profonde per evitare riferimenti condivisi
validation_data_aux = copy.deepcopy(test_data)

# Sostituisci i valori del campo 'labels' di validation_data_aux con quelli di aux
for i, element in enumerate(validation_data_aux):
    element["labels"] = y_pred[i]




In [ ]:
# Salva il nuovo validation_data modificato in un file pred.json
output_file_path = "pred4.json"
with open(output_file_path, "w", encoding='utf-8') as output_file:
    json.dump(validation_data_aux, output_file, indent=2)


# python your_script.py --gold_file_path /path/to/gold_file.txt --pred_file_path /path/to/pred_file.txt
# subtask_1_2a.py --gold_file_path ./subtask1/validation.json --pred_file_path ./subtask1/pred.json
# python subtask_1_2a.py --gold_file_path ./subtask1/dev_subtask1_en.json --pred_file_path ./subtask1/pred41.json

In [ ]:
import json
import copy

# Funzione per sostituire le istanze di "\u00e9" con "é"
def replace_unicode(data):
    if isinstance(data, str):
        return data.replace('\u00e9', 'é')
    elif isinstance(data, list):
        return [replace_unicode(item) for item in data]
    elif isinstance(data, dict):
        return {key: replace_unicode(value) for key, value in data.items()}
    else:
        return data

# Creazione di copie profonde per evitare riferimenti condivisi
validation_data_aux = copy.deepcopy(test_vero_data)

# Sostituisci i valori del campo 'labels' di validation_data_aux con quelli di aux
for i, element in enumerate(validation_data_aux):
    element["labels"] = y_pred[i]

# Applica la sostituzione alle etichette nel formato richiesto
formatted_data = [replace_unicode(item) for item in validation_data_aux]

# Salva il nuovo validation_data modificato in un file pred.json
output_file_path = "pred6.json"
with open(output_file_path, "w", encoding='utf-8') as output_file:
    output_file.write(json.dumps(formatted_data, indent=2, separators=(',', ':')))


 ############################################## INIZIO SEZIONE TEST #################################

In [ ]:
test = Free_classifier.predict_proba(X_test[160])
print(test)

[array([[0.5, 0.5]]), array([[0.5, 0.5]])]


In [ ]:
labels_to_check = ["Free", "Persuasion"]
X, Y = filter_matrix_by_labels(X_test, y_test, labels_to_check, G)

new_Y = transform_Y_with_ancestors(G, Y, labels_to_check)
print(Y)
print(new_Y)
mlb = MultiLabelBinarizer()
Y = Binarizzatore_etichette(labels_to_check, new_Y)

[['Causal Oversimplification'], ['Smears', 'Name calling/Labeling', 'Black-and-white Fallacy/Dictatorship', 'Loaded Language'], ['Smears', "Misrepresentation of Someone's Position (Straw Man)"], ['Free'], ['Loaded Language'], ['Bandwagon'], ['Smears', 'Black-and-white Fallacy/Dictatorship'], ['Appeal to authority'], ['Appeal to fear/prejudice', 'Black-and-white Fallacy/Dictatorship', 'Loaded Language'], ['Smears', 'Causal Oversimplification', 'Name calling/Labeling', 'Exaggeration/Minimisation'], ['Glittering generalities (Virtue)'], ['Appeal to authority', 'Whataboutism'], ['Smears', 'Loaded Language', 'Exaggeration/Minimisation'], ['Loaded Language'], ['Name calling/Labeling'], ['Name calling/Labeling', 'Appeal to fear/prejudice', 'Black-and-white Fallacy/Dictatorship', 'Loaded Language'], ['Smears', 'Name calling/Labeling'], ['Black-and-white Fallacy/Dictatorship', 'Slogans'], ['Appeal to authority', 'Loaded Language'], ['Free'], ['Slogans'], ['Smears', 'Black-and-white Fallacy/Dict

In [ ]:
with open('output.txt', 'w') as file:
    line = str(y_pred)  # Converti la lista di liste in una stringa
    file.write(f"{line}\n")


In [ ]:
def find_descendants(graph, labels):
    descendants = set()

    def dfs(node):
        descendants.add(node)
        [dfs(child) for child in graph.successors(node)]

    [dfs(label) for label in labels]
    return descendants

def filter_matrix_by_labels(X, y, labels, graph):
    descendants_set = find_descendants(graph, labels)

    filtered_X, filtered_labels = zip(*[(X[i], list(set(labels_list) & descendants_set)) for i, labels_list in enumerate(y) if set(labels_list) & descendants_set])

    return np.array(filtered_X), list(filtered_labels)

def find_ancestor_in_labels(graph, node, target_labels):
    ancestors = set()

    def dfs(current):
        ancestors.add(current)
        [dfs(pred) for pred in graph.predecessors(current)]

    [dfs(predecessor) for predecessor in graph.predecessors(node)]

    common_ancestors = ancestors & set(target_labels)
    return list(common_ancestors)[0] if common_ancestors else None

def transform_Y_with_ancestors(graph, Y, target_labels):
    return [list(set([find_ancestor_in_labels(graph, label, target_labels) or label for label in labels_list])) for labels_list in Y]

labels_to_check = ["Ethos", "Pathos", "Logos"]
labels_to_check = ["Whataboutism","Misrepresentation of Someone's Position (Straw Man)","Presenting Irrelevant Data (Red Herring)"]
labels_to_check = ["Ad Hominem", "Bandwagon", "Glittering generalities (Virtue)" , "Appeal to authority"]

labels_to_check = ["Repetition", "Obfuscation, Intentional vagueness, Confusion", "Reasoning", "Justification"]
labels_to_check = ["Free", "Persuasion"]
X, Y = filter_matrix_by_labels(X_train, y_train, labels_to_check, G)

new_Y = transform_Y_with_ancestors(G, Y, labels_to_check)
print(Y)
print(new_Y)


In [ ]:
import json

json_file_path = 'pred4.json'
txt_file_path = 'output.txt'

# Leggi i dati dal file JSON
with open(json_file_path, 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)

# Prepara i dati nel formato richiesto
formatted_data = []
for item in data:
    # Sostituisci 'labels' con il campo appropriato da y_pred
    labels = item.get("labels", [])
    formatted_labels = [f'"{label}"' for label in labels]
    formatted_item = {"id": item.get("id", ""), "labels": formatted_labels}
    formatted_data.append(formatted_item)

# Scrivi i dati formattati nel file di testo con virgolette tra le labels, su una singola riga
with open(txt_file_path, 'w', encoding='utf-8') as txt_file:
    txt_file.write(json.dumps(formatted_data, separators=(',', ':')))






In [ ]:
from collections import Counter

import json
import os
import numpy as np
import networkx as nx

from networkx import DiGraph
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix
import re
def read_json_file(file_path):
    if os.path.exists(file_path):
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
        return data
    else:
        return None
# Leggi train.json
train_data = read_json_file("train.json")

# Leggi validation.json
validation_data = read_json_file("validation.json")

test_data = read_json_file("dev_subtask1_en.json")


# Inizializza le liste per i dati di addestramento
texts_train, labels_train = [], []
texts_validation, labels_validation = [], []
texts_test, labels_test = [], []
texts_test_vero, labels_test_vero = [], []
if train_data:
    texts_train = [element['text'] for element in train_data]
    labels_train = [element['labels'] if element['labels'] else [] for element in train_data]

# Estre x e y dai dataset se esistono
if validation_data:
    texts_validation = [element['text'] for element in validation_data]
    labels_validation = [element['labels'] if element['labels'] else [] for element in validation_data]

if test_data:
    texts_test = [element['text'] for element in test_data]
    labels_test = [element['labels'] if element['labels'] else [] for element in test_data]




In [ ]:
# Dopo aver creato e popolato tutte le liste y_pred
element_count = Counter(len(lst) for lst in labels_test)

for num_elements in range(11):
    count = element_count.get(num_elements, 0)
    print(f"Numero di liste con {num_elements} elemento/i: {count}")

Numero di liste con 0 elemento/i: 156
Numero di liste con 1 elemento/i: 313
Numero di liste con 2 elemento/i: 251
Numero di liste con 3 elemento/i: 169
Numero di liste con 4 elemento/i: 79
Numero di liste con 5 elemento/i: 25
Numero di liste con 6 elemento/i: 4
Numero di liste con 7 elemento/i: 2
Numero di liste con 8 elemento/i: 1
Numero di liste con 9 elemento/i: 0
Numero di liste con 10 elemento/i: 0


In [ ]:
# Install required packages
!pip install networkx

import json
import networkx as nx

# File paths
gold_file_path = 'dev_subtask1_en.json'
pred_file_path = 'pred.json'

# Load JSON files
with open(gold_file_path, 'r') as file:
    gold_data = json.load(file)

with open(pred_file_path, 'r') as file:
    pred_data = json.load(file)

# Create the directed graph
G = nx.DiGraph()
G.add_edge('ROOT', 'Free')
G.add_edge('ROOT', 'Persuasion')
G.add_edge('Persuasion', 'Logos')
G.add_edge('Logos', 'Repetition')
G.add_edge('Logos', 'Obfuscation, Intentional vagueness, Confusion')
G.add_edge('Logos', 'Reasoning')
G.add_edge('Logos', 'Justification')
G.add_edge('Justification', 'Slogans')
G.add_edge('Justification', 'Bandwagon')
G.add_edge('Justification', 'Appeal to authority')
G.add_edge('Justification', 'Flag-waving')
G.add_edge('Justification', 'Appeal to fear/prejudice')
G.add_edge('Reasoning', 'Simplification')
G.add_edge('Simplification', 'Causal Oversimplification')
G.add_edge('Simplification', 'Black-and-white Fallacy/Dictatorship')
G.add_edge('Simplification', 'Thought-terminating cliché')
G.add_edge('Reasoning', 'Distraction')
G.add_edge('Distraction', "Misrepresentation of Someone's Position (Straw Man)")
G.add_edge('Distraction', 'Presenting Irrelevant Data (Red Herring)')
G.add_edge('Distraction', 'Whataboutism')
G.add_edge('Persuasion', 'Ethos')
G.add_edge('Ethos', 'Appeal to authority')
G.add_edge('Ethos', 'Glittering generalities (Virtue)')
G.add_edge('Ethos', 'Bandwagon')
G.add_edge('Ethos', 'Ad Hominem')
G.add_edge('Ad Hominem', 'Doubt')
G.add_edge('Ad Hominem', 'Name calling/Labeling')
G.add_edge('Ad Hominem', 'Smears')
G.add_edge('Ad Hominem', 'Reductio ad hitlerum')
G.add_edge('Ad Hominem', 'Whataboutism')
G.add_edge('Persuasion', 'Pathos')
G.add_edge('Pathos', 'Exaggeration/Minimisation')
G.add_edge('Pathos', 'Loaded Language')
G.add_edge('Pathos', 'Appeal to fear/prejudice')
G.add_edge('Pathos', 'Flag-waving')

# Function to check if one label is an ancestor of another
def is_ancestor(G, ancestor, descendant):
    return nx.has_path(G, ancestor, descendant)

# Initialize counters
completely_correct = 0
partially_correct = 0
completely_wrong = 0
leaf_label_stats = {}

# Get leaf labels
leaf_labels = [node for node in G.nodes() if G.out_degree(node) == 0]

# Initialize leaf label stats
for label in leaf_labels:
    leaf_label_stats[label] = {'total': 0, 'correct': 0}

# Compare predictions to gold labels
for gold_item, pred_item in zip(gold_data, pred_data):
    gold_labels = gold_item['labels']
    pred_labels = pred_item['labels']

    gold_set = set(gold_labels)
    pred_set = set(pred_labels)

    if gold_set == pred_set:
        completely_correct += 1
    elif any(any(is_ancestor(G, pred, gold) for pred in pred_set) for gold in gold_set):
        partially_correct += 1
    else:
        completely_wrong += 1

    for gold_label in gold_labels:
        if gold_label in leaf_label_stats:
            leaf_label_stats[gold_label]['total'] += 1
            if gold_label in pred_labels:
                leaf_label_stats[gold_label]['correct'] += 1

# Print results
print(f"Completely correct: {completely_correct}")
print(f"Partially correct: {partially_correct}")
print(f"Completely wrong: {completely_wrong}")

print("\nLeaf label statistics:")
for label, stats in leaf_label_stats.items():
    print(f"{label}: {stats['correct']}/{stats['total']} correct")


Completely correct: 96
Partially correct: 495
Completely wrong: 409

Leaf label statistics:
Free: 0/0 correct
Repetition: 1/46 correct
Obfuscation, Intentional vagueness, Confusion: 0/8 correct
Slogans: 27/111 correct
Bandwagon: 1/16 correct
Appeal to authority: 68/136 correct
Flag-waving: 16/89 correct
Appeal to fear/prejudice: 3/66 correct
Causal Oversimplification: 1/53 correct
Black-and-white Fallacy/Dictatorship: 48/98 correct
Thought-terminating cliché: 3/78 correct
Misrepresentation of Someone's Position (Straw Man): 0/10 correct
Presenting Irrelevant Data (Red Herring): 0/10 correct
Whataboutism: 1/52 correct
Glittering generalities (Virtue): 5/71 correct
Doubt: 0/45 correct
Name calling/Labeling: 88/262 correct
Smears: 221/282 correct
Reductio ad hitlerum: 0/11 correct
Exaggeration/Minimisation: 4/62 correct
Loaded Language: 121/303 correct


In [ ]:
# Install required packages
!pip install networkx

import json
import networkx as nx

# File paths
gold_file_path = 'dev_subtask1_en.json'
pred_file_path = 'pred.json'

# Load JSON files
with open(gold_file_path, 'r') as file:
    gold_data = json.load(file)

with open(pred_file_path, 'r') as file:
    pred_data = json.load(file)

# Create the directed graph
G = nx.DiGraph()
G.add_edge('ROOT', 'Free')
G.add_edge('ROOT', 'Persuasion')
G.add_edge('Persuasion', 'Logos')
G.add_edge('Logos', 'Repetition')
G.add_edge('Logos', 'Obfuscation, Intentional vagueness, Confusion')
G.add_edge('Logos', 'Reasoning')
G.add_edge('Logos', 'Justification')
G.add_edge('Justification', 'Slogans')
G.add_edge('Justification', 'Bandwagon')
G.add_edge('Justification', 'Appeal to authority')
G.add_edge('Justification', 'Flag-waving')
G.add_edge('Justification', 'Appeal to fear/prejudice')
G.add_edge('Reasoning', 'Simplification')
G.add_edge('Simplification', 'Causal Oversimplification')
G.add_edge('Simplification', 'Black-and-white Fallacy/Dictatorship')
G.add_edge('Simplification', 'Thought-terminating cliché')
G.add_edge('Reasoning', 'Distraction')
G.add_edge('Distraction', "Misrepresentation of Someone's Position (Straw Man)")
G.add_edge('Distraction', 'Presenting Irrelevant Data (Red Herring)')
G.add_edge('Distraction', 'Whataboutism')
G.add_edge('Persuasion', 'Ethos')
G.add_edge('Ethos', 'Appeal to authority')
G.add_edge('Ethos', 'Glittering generalities (Virtue)')
G.add_edge('Ethos', 'Bandwagon')
G.add_edge('Ethos', 'Ad Hominem')
G.add_edge('Ad Hominem', 'Doubt')
G.add_edge('Ad Hominem', 'Name calling/Labeling')
G.add_edge('Ad Hominem', 'Smears')
G.add_edge('Ad Hominem', 'Reductio ad hitlerum')
G.add_edge('Ad Hominem', 'Whataboutism')
G.add_edge('Persuasion', 'Pathos')
G.add_edge('Pathos', 'Exaggeration/Minimisation')
G.add_edge('Pathos', 'Loaded Language')
G.add_edge('Pathos', 'Appeal to fear/prejudice')
G.add_edge('Pathos', 'Flag-waving')

# Function to check if one label is an ancestor of another
def is_ancestor(G, ancestor, descendant):
    return nx.has_path(G, ancestor, descendant)

# Initialize counters
completely_correct = 0
partially_correct = 0
completely_wrong = 0
leaf_label_stats = {}
correct_free_predictions = 0
total_free_gold = 0

# Get leaf labels
leaf_labels = [node for node in G.nodes() if G.out_degree(node) == 0]

# Initialize leaf label stats
for label in leaf_labels:
    leaf_label_stats[label] = {'total': 0, 'correct': 0}

# Compare predictions to gold labels
for gold_item, pred_item in zip(gold_data, pred_data):
    gold_labels = gold_item['labels']
    pred_labels = pred_item['labels']

    if not gold_labels:
        total_free_gold += 1
        if not pred_labels:
            correct_free_predictions += 1
        continue  # Skip further checks for "Free" labels

    gold_set = set(gold_labels)
    pred_set = set(pred_labels)

    if gold_set == pred_set:
        completely_correct += 1
    elif any(any(is_ancestor(G, pred, gold) for pred in pred_set) for gold in gold_set):
        partially_correct += 1
    else:
        completely_wrong += 1

    for gold_label in gold_labels:
        if gold_label in leaf_label_stats:
            leaf_label_stats[gold_label]['total'] += 1
            if gold_label in pred_labels:
                leaf_label_stats[gold_label]['correct'] += 1

# Print results
print(f"Completely correct: {completely_correct}")
print(f"Partially correct: {partially_correct}")
print(f"Completely wrong: {completely_wrong}")
print(f"Correct 'Free' predictions: {correct_free_predictions}/{total_free_gold}")

print("\nLeaf label statistics:")
for label, stats in leaf_label_stats.items():
    print(f"{label}: {stats['correct']}/{stats['total']} correct")


Completely correct: 54
Partially correct: 495
Completely wrong: 295
Correct 'Free' predictions: 42/156

Leaf label statistics:
Free: 0/0 correct
Repetition: 1/46 correct
Obfuscation, Intentional vagueness, Confusion: 0/8 correct
Slogans: 27/111 correct
Bandwagon: 1/16 correct
Appeal to authority: 68/136 correct
Flag-waving: 16/89 correct
Appeal to fear/prejudice: 3/66 correct
Causal Oversimplification: 1/53 correct
Black-and-white Fallacy/Dictatorship: 48/98 correct
Thought-terminating cliché: 3/78 correct
Misrepresentation of Someone's Position (Straw Man): 0/10 correct
Presenting Irrelevant Data (Red Herring): 0/10 correct
Whataboutism: 1/52 correct
Glittering generalities (Virtue): 5/71 correct
Doubt: 0/45 correct
Name calling/Labeling: 88/262 correct
Smears: 221/282 correct
Reductio ad hitlerum: 0/11 correct
Exaggeration/Minimisation: 4/62 correct
Loaded Language: 121/303 correct
